# Field Completeness by Accession Decade

In [1]:
import glob
import pandas as pd

objects_df = pd.read_hdf('/home/richard/Git/bbk-mqaf/data/latest/field.h5')

In [3]:
# First create new data frame reducing down to just the columns we need

complete_df = objects_df.filter(regex='accessionYear|completeness*', axis=1)

,accessionYear,completeness:TOTAL,completeness:COMPOSITION,completeness:PRODUCTION,completeness:ASSOCIATION,completeness:MATERIALITY,completeness:MUSEUM,completeness:HISTORY,completeness:PHYSICALITY,completeness:ACADEMIC,completeness:DEPICTION
0,2012.0,0.435897,1.0,0.714286,0.000000,1.0,1.0,0.4,0.75,0.0,0.222222
1,2012.0,0.333333,1.0,0.714286,0.166667,1.0,1.0,0.2,0.50,0.0,0.000000
2,2012.0,0.307692,1.0,0.714286,0.000000,1.0,1.0,0.2,0.50,0.0,0.000000
3,2012.0,0.410256,1.0,0.714286,0.000000,1.0,1.0,0.2,0.75,0.0,0.222222
4,2012.0,0.358974,1.0,0.714286,0.000000,1.0,1.0,0.2,0.50,0.0,0.222222
...,...,...,...,...,...,...,...,...,...,...,...
8759,1990.0,0.333333,1.0,0.714286,0.000000,1.0,1.0,0.0,0.50,0.0,0.222222
8760,1990.0,0.333333,1.0,0.714286,0.000000,1.0,1.0,0.0,0.50,0.0,0.222222
8761,1990.0,0.333333,1.0,0.714286,0.000000,1.0,1.0,0.0,0.50,0.0,0.222222
8762,1990.0,0.333333,1.0,0.714286,0.000000,1.0,1.0,0.0,0.50,0.0,0.222222


In [4]:
import re
tidied_df = complete_df.rename(columns=lambda x: re.sub('completeness:','',x))

,accessionYear,TOTAL,COMPOSITION,PRODUCTION,ASSOCIATION,MATERIALITY,MUSEUM,HISTORY,PHYSICALITY,ACADEMIC,DEPICTION
0,2012.0,0.435897,1.0,0.714286,0.000000,1.0,1.0,0.4,0.75,0.0,0.222222
1,2012.0,0.333333,1.0,0.714286,0.166667,1.0,1.0,0.2,0.50,0.0,0.000000
2,2012.0,0.307692,1.0,0.714286,0.000000,1.0,1.0,0.2,0.50,0.0,0.000000
3,2012.0,0.410256,1.0,0.714286,0.000000,1.0,1.0,0.2,0.75,0.0,0.222222
4,2012.0,0.358974,1.0,0.714286,0.000000,1.0,1.0,0.2,0.50,0.0,0.222222
...,...,...,...,...,...,...,...,...,...,...,...
8759,1990.0,0.333333,1.0,0.714286,0.000000,1.0,1.0,0.0,0.50,0.0,0.222222
8760,1990.0,0.333333,1.0,0.714286,0.000000,1.0,1.0,0.0,0.50,0.0,0.222222
8761,1990.0,0.333333,1.0,0.714286,0.000000,1.0,1.0,0.0,0.50,0.0,0.222222
8762,1990.0,0.333333,1.0,0.714286,0.000000,1.0,1.0,0.0,0.50,0.0,0.222222


## Decades Completeness Overview

In [6]:
# Save all the possible collectioncode
import altair as alt

ranges = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
count =0

dept_col_counts = None
charts = []
hcharts = []

def create_chart(data_df, title):
  data_rows_df = pd.DataFrame()
  for column in data_df.columns[1:]:
    # Group into counts of percentages 0-10, 10-20, ... 90-100
    # dept_col_counts = dept_stats_df[column].value_counts(bins=10, sort=False)
    
    dept_col_counts = data_df[column].groupby(pd.cut(data_df[column], ranges, labels=["0%","10%","20%","30%","40%","50%","60%","70%","80%","90%"], include_lowest=True)).count()
    dept_col_counts.name = column
    data_rows_df = data_rows_df.append(dept_col_counts, ignore_index=False)
  data_rows_df = data_rows_df.rename_axis('Concept').rename_axis('Percentages', axis='columns')
    # TO handle converting from CategoricalIndex - may change https://github.com/pandas-dev/pandas/issues/19136
  data_rows_df.columns = data_rows_df.columns.tolist()
  data_rows_df = data_rows_df.reset_index()
  data_rows_melt_df = data_rows_df.melt(id_vars=['Concept'])
  chart = alt.Chart(data_rows_melt_df).mark_bar().encode(
       x='value:Q',
       y='Concept:O',
        color='variable:O'
    ).properties(width=220, title=title)
  
  return chart
        
for decade in range(1850,2030,10):
  dept_rows_df = pd.DataFrame()
  dept_stats_df = tidied_df[tidied_df[' accessionYear'].between(decade,decade+9)] 
  # Now need to loop over each completeness coll in turn
  for column in dept_stats_df.columns[1:]:
    # Group into counts of percentages 0-10, 10-20, ... 90-100
    # dept_col_counts = dept_stats_df[column].value_counts(bins=10, sort=False)
    
    dept_col_counts = dept_stats_df[column].groupby(pd.cut(dept_stats_df[column], ranges, labels=["0%","10%","20%","30%","40%","50%","60%","70%","80%","90%"], include_lowest=True)).count()
    dept_col_counts.name = column
    dept_rows_df = dept_rows_df.append(dept_col_counts, ignore_index=False)
    
  dept_rows_df = dept_rows_df.rename_axis('Concept').rename_axis('Percentages', axis='columns')
    # TO handle converting from CategoricalIndex - may change https://github.com/pandas-dev/pandas/issues/19136
  dept_rows_df.columns = dept_rows_df.columns.tolist()
  #print(dept_rows_df)
  dept_rows_df = dept_rows_df.reset_index()

  dept_rows_melt_df = dept_rows_df.melt(id_vars=['Concept'])

  chart = alt.Chart(dept_rows_melt_df).mark_bar().encode(
       x='value:Q',
       y='Concept:O',
        color='variable:O',
      tooltip=['variable', 'value']
    ).properties(width=220, title="%s (%d)" % (decade, len(dept_stats_df)))
  charts.append(chart)
  if count > 1:
      hcharts.append(alt.hconcat(*charts))
      charts = []
      count = 0
  else:
      count += 1

# Handle post current year, no year and less than opening year (1857)

dept_stats_df = tidied_df[tidied_df[' accessionYear'] < 1857.0] 
#print(dept_stats_df)
charts.append(create_chart(dept_stats_df, "Pre-1857 (%d)" % len(dept_stats_df)))
      
dept_stats_df = tidied_df[tidied_df[' accessionYear'].isnull()] 
#print(dept_stats_df)
charts.append(create_chart(dept_stats_df, "No Year (%d)" % len(dept_stats_df)))
    
hcharts.append(alt.hconcat(*charts))

alt.vconcat(*hcharts)


alt.VConcatChart(...)